In [8]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
#nltk.download('perluniprops')
from nltk import ngrams
from itertools import chain

import pickle

## Fetch even data

In [9]:
with open('X_train_even.pickle', 'rb') as handle:
    X_train_even = pickle.load(handle)
    
# with open('X_test_even.pickle', 'rb') as handle:
#     X_test_even = pickle.load(handle)
    
with open('Y_train_even.pickle', 'rb') as handle:
    Y_train_even = pickle.load(handle)
    
# with open('Y_test_even.pickle', 'rb') as handle:
#     Y_test_even = pickle.load(handle)

print('Total number of even training data points: %d' %len(X_train_even))

Total number of even training data points: 199483


## First model on even data

In [10]:
# Add your import statements here
from keras import Input, Model
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, Embedding
from keras.layers.recurrent import LSTM
from keras.callbacks import TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential

def create_model(X_seq_len, output_n, hidden_dim, vocabulary_size, embedding_dim):

    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, input_length=X_seq_len))
    model.add(LSTM(hidden_dim))
    model.add(Dense(output_n,activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

#### Create a smaller data set for testing

In [11]:
lesser_number = 10000

small_Y = Y_train_even[0:lesser_number]
small_X = X_train_even[0:lesser_number]
neg_share = len(small_Y[small_Y==1])/lesser_number

print('The share of negative reviews:')
print(neg_share)
print('It is approximately even!')

The share of negative reviews:
0.4924
It is approximately even!


#### Set the data to train for 

In [16]:
X_train_here = X_train_even
Y_train_here = Y_train_even

#### Flatten input data for embedding layer

In [17]:
dim1 = len(X_train_here)
dim2 = len(X_train_here[0])

X_train_here_flat = X_train_here.reshape(dim1,dim2)

### Train!

In [18]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 10
num_epochs = 10
b_size = 64

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 10

model = create_model(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_even_data_more'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 10)           462110    
_________________________________________________________________
lstm_6 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 462,961
Trainable params: 462,961
Non-trainable params: 0
_________________________________________________________________
Train on 159586 samples, validate on 39897 samples
Epoch 1/10
 16064/159586 [==>...........................] - ETA: 12:25 - loss: 0.6856 - acc: 0.5361

KeyboardInterrupt: 